# Making Transformers Efficient in Production

In previous notebooks, we've seen how transformers can be fine-tuned for various tasks. However in many situations irrespective of the metric, model is not very useful if it's too slow or too large to meet the buisness requirements of the application. The obvious alternative is to train a faster and more compact model, but with reduction comes the degradation in performance. What if we need a compact yet highly accurate model?

In this notebook we'll cover four techniques with Open Neural Network Exchange(ONNX) and ONNX Runtime(ORT) to reduce the prediction time and memory footprint of the transformers, they are:
    1. *Knowledge distillization*
    2. *Quantization*
    3. *Pruning*
    4. *Graph Optimization*

We'll also see how the techniques can be combined to produce significant performance gains. [How Roblox engineering team scaled Bert to serve 1+ Billion Daily Requests on CPUs](https://medium.com/@quocnle/how-we-scaled-bert-to-serve-1-billion-daily-requests-on-cpus-d99be090db26) and found that knowledge distillization and quantization improved the latency and throughput of their BERT classifier over a factor of 30!

To illustrate the benefits and tradeoffs associated with each technique, we'll use intent detection(important component of text-based assistants), where low latency is critical for maintaining a conversation in real time.
Along the way, we'll also learn how to create custom trainers and perform hyperparamter search, and gain a sense of what it takes to implement cutting-edge research with Transformers(lib).

## Intent Detection as a Case Study

Let's suppose we're trying to build a text-based assistant for a company's call center to deal with banking and bookings without human interaction. Base on a wide variety of natural language text(input from user), our assistant needs to classify the input into a set of predefiend actions or *intents*.

For example:

*Message*: Hey, i'd like to rent a vehichle on Nov1st.

The assitant will classify this as *car rental intent* which then triggers an action and an response.

The assistance must also have the ability to identify out of scope(oos) intents to be robust in a production environment.

As a baseline, a BERT-model has been fine-tuned with accuracy around 94% on [CLINIC150 dataset](https://arxiv.org/abs/1909.02027). This dataset includes 22,500 inscope queries across 150 intents and 10 domains like banking and travel, and also includes 1,200 *oos* intent class.

> **Note:** In practive, we would gather in-house dataset, but using public data is a greate way to iterate quickly and generate preliminary results.

First let's load the model and wrap it around a text-classification pipeline.

In [1]:
from transformers import pipeline

bert_ckpt = "transformersbook/bert-base-uncased-finetuned-clinc"
pipe = pipeline(
    task="text-classification",
    model=bert_ckpt
)

2023-09-20 08:23:10.059730: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


: 